
# **Решение методом Гаусса с выбором главного элемента**

In [1]:
import numpy as np
import numba
import random 
import time
import pandas as pd
'''
Декоратор numba.njit предписывает транслировать функцию в чистый машинный код, который не обращается к интерпретатору пайтона.
В общем случае это невозможно, но в данном случае у нас все вычисления идут только с numpy,
а для этого пакета numba умеет вызывать Си-инетерфейсы для соответствующих операций - индексирования, присваивания, арифметики.
'''

'\nДекоратор numba.njit предписывает транслировать функцию в чистый машинный код, который не обращается к интерпретатору пайтона.\nВ общем случае это невозможно, но в данном случае у нас все вычисления идут только с numpy,\nа для этого пакета numba умеет вызывать Си-инетерфейсы для соответствующих операций - индексирования, присваивания, арифметики.\n'

In [5]:
@numba.njit
def makeTrianglePivot(matrix):
    buf = np.zeros(matrix.shape[1])

    for nrow in range(len(matrix)):
        pivot = nrow + np.argmax(np.abs(matrix[nrow:, nrow]))
        if pivot != nrow:
            matrix[nrow], matrix[pivot] = matrix[pivot], np.copy(matrix[nrow])
        row = matrix[nrow]
        divider = row[nrow] # диагональный элемент
        if abs(divider) < 1e-10:
            raise ValueError("Матрица несовместна")
        row[nrow:] *= 1/divider
        row[nrow] = 1.0
        for lr in range(nrow+1, len(matrix)):
            lower_row = matrix[lr]
            factor = lower_row[nrow]
            np.multiply(factor, row, buf)
            lower_row -= buf
    return matrix

In [6]:
@numba.njit
def makeIdentity(matrix):
    N = matrix.shape[0]
    matrix = matrix.T
    roots = matrix[-1]
    for nrow in range(N-1,0,-1):
        root = roots[nrow]
        column = matrix[nrow]
        roots[:nrow] -= root*column[:nrow]
        column[:nrow] = 0.0
        
    return matrix.T

In [2]:
def gaussSolvePivot(A, b=None):
    start_time = time.time()
    """Решает систему линейных алгебраических уравнений Ax=b
    Если b is None, то свободные коэффициенты в последней колонке"""
    shape = A.shape
    assert len(shape) == 2, ("Матрица не двумерная", shape) # двумерная матрица
    A = A.copy()
    if b is not None:
        assert shape[0] == shape[1], ("Матрица не квадратная", shape)
        assert b.shape[0] == shape[0], ("Размерность свободных членов не соответствует матрице", shape, b.shape)
        # добавляем свободные члены дополнительным столбцом
        A = np.c_[A, b]
    else:
        # Проверяем, что квадратная плюс столбец
        assert shape[0]+1 == shape[1], ("Неверный формат матрицы", shape)
    makeTrianglePivot(A)
    makeIdentity(A)
    return A[:,-1], time.time() - start_time

# **Генератор Матриц**

In [27]:
def generator(size, t = 3, lower_bound = -10, upper_bound = 10): # t - количество чисел после запятой
  randomSle = np.round(np.random.uniform(low=lower_bound,high=upper_bound,size=(size,size)),t)
  randomV = np.round(np.random.uniform(low=lower_bound,high=upper_bound,size=(size,1)),t)
  return (randomSle, randomV)

# **Тесты**

In [36]:
table = pd.DataFrame()
a = [5,10,20,30,50,100,200,500,1000,5000]
for i in a:
  time_tmp = []
  er_tmp = []
  er_ot = []
  for j in range(3):
    matrix = generator(i,t=2,lower_bound=-100,upper_bound=100)
    result = gaussSolvePivot(*matrix)
    time_tmp.append(result[1])

    np_roots = np.linalg.solve(*matrix)
    tmp = np.abs(np.matmul(matrix[0], np_roots) - matrix[1])
    er_tmp.append(tmp)
    er_ot.append(np.abs(np.matmul(matrix[0], np_roots) - matrix[1])/(matrix[1]) * 100)
  table = table.append({'Размерность матрицы':int(i),'Среднее время выполнения(сек,5 знаков)':round(np.mean(time_tmp),5) ,
                        'Средняя Абсолютная погрешность (10 знаков)': round(np.mean(er_tmp),10),
                        'Средняя Относительная погрешность': np.mean(er_ot)},ignore_index=True)

print(table.head())

<ipython-input-36-cfa77759e900>:15: RuntimeWarning: divide by zero encountered in true_divide
  er_ot.append(np.abs(np.matmul(matrix[0], np_roots) - matrix[1])/(matrix[1]) * 100)


   Размерность матрицы  Среднее время выполнения(сек,5 знаков)  \
0                  5.0                                 0.00009   
1                 10.0                                 0.00006   
2                 20.0                                 0.00010   
3                 30.0                                 0.00015   
4                 50.0                                 0.00021   

   Средняя Абсолютная погрешность (10 знаков)  \
0                                         0.0   
1                                         0.0   
2                                         0.0   
3                                         0.0   
4                                         0.0   

   Средняя Относительная погрешность  
0                      -4.540787e-14  
1                      -1.318590e-14  
2                      -3.232703e-13  
3                       1.249342e-13  
4                       2.025518e-12  
